In [9]:
import numpy as np 
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.layers import Dropout
from keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
train_path = 'E:\datasets\train'
valid_path = 'E:\datasets\val'
test_path = 'E:\datasets\test'

In [7]:
cd E:\datasets

E:\datasets


In [12]:
def preprocess(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #img = tf.keras.preprocessing.image.img_to_array(img,dtype=np.uint8)
    #img = np.expand_dims(img, axis=0)
    img = np.array(img,dtype=np.uint8)
    img = cv2.GaussianBlur(img, (3, 3), 7)
    mean = int(np.mean(img))
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
            img[i,j] = max(0,img[i,j]-mean)
    clahe = cv2.createCLAHE()
    img = clahe.apply(img)
    img = img.reshape(-1, 224, 224, 1)
    return img/255.0

#Image loading and Preprocessing 
train_batches = ImageDataGenerator(preprocessing_function=preprocess).flow_from_directory(train_path, target_size=(224,224), classes=['0', '1','2','3','4'], batch_size=10,shuffle=True)
valid_batches = ImageDataGenerator(preprocessing_function=preprocess).flow_from_directory(valid_path, target_size=(224,224), classes=['0', '1','2','3','4'], batch_size=10,shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=preprocess).flow_from_directory(test_path, target_size=(224,224), classes=['0', '1','2','3','4'], batch_size=10,shuffle=False)

Found 0 images belonging to 5 classes.
Found 0 images belonging to 5 classes.
Found 0 images belonging to 5 classes.


In [6]:
cd E:\knee_OA_dl_app

E:\knee_OA_dl_app


In [13]:
import matplotlib.pyplot as plt
# plots images with labels within jupyter notebook
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
imgs, labels = next(train_batches)
plots(imgs, titles=labels)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [1]:
#train_batches = train_batches.reshape(-1, 224, 224, 1)
imgs, labels = next(train_batches)
print(imgs[0].shape)
#print(type(train_batches))

NameError: name 'train_batches' is not defined

In [ ]:
#Regularized Model
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.3))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.3))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Flatten())
model.add(Dense(units=5, activation="softmax"))
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Extensive
from tensorflow.keras.layers import Dropout
from keras import regularizers
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.1)))
model.add(Dropout(0.5))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.1)))
model.add(Dropout(0.5))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.1)))
model.add(Dropout(0.5))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.1)))
model.add(Dropout(0.5))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.1)))
model.add(Dropout(0.5))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Flatten())
model.add(Dense(units=5, activation="softmax"))
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [1]:
model.fit(train_batches, batch_size=10 , validation_data=valid_batches, epochs=13, verbose=2)           

NameError: name 'model' is not defined

In [ ]:
#2
model.save('dataset/OF_2.h5')

In [ ]:
predictions = model.predict(test_batches, batch_size=10, verbose=0)

In [ ]:
Grade_predictions = np.argmax(predictions,axis=-1)

In [ ]:
print(predictions)

In [ ]:
for i in Grade_predictions:
    print(i)

In [ ]:
test_labels = test_batches[1]

In [ ]:
cm = confusion_matrix(test_labels, Grade_predictions)